In [4]:

def set_course():
    import current_course as cc
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    from requests import HTTPError

    from casstools.cass_client import CassClient

    client = CassClient()
    courses = sorted([c['object_name'].replace("/", "") for c in client.get_courses()])

    course = cc.read_current_course()
    if course is None:
        course = courses[0]
    course_dropdown = widgets.Dropdown(
        options=courses,
        value=course,
        description='Course:',
        layout={'width': 'max-content'},
        disabled=False
    )

    tabs = widgets.ToggleButtons(
        options=['Assignments', 'Roster'],
        description='',
        disabled=False,
        button_style='',
        tooltips=['View Assignments', 'View Roster'],
        layout={'width': 'max-content'}
    )

    def course_dropdown_on_change(*args):
        tabs_on_click(*args)
        cc.write_current_course(course_dropdown.value)

    def tabs_on_click(*args):
        with output:
            clear_output()
            display(course_dropdown.value)
            if tabs.index == 0:
                try:
                    a_df = client.get_course_assignments(course_dropdown.value)
                    display(a_df)
                except HTTPError:
                    display("No Assignments File uploaded")
            else:
                try:
                    r_df = client.get_course_roster(course_dropdown.value)
                    display(r_df)
                except HTTPError:
                    display("No Roster File uploaded")

    # handlers
    course_dropdown.observe(course_dropdown_on_change)
    tabs.observe(tabs_on_click)
    output = widgets.Output()

    # layouts
    display(HTML("<h3>Set Current Course</h3>"))
    display(course_dropdown)
    display(tabs)
    display(output)

    # initialize
    course_dropdown_on_change()
    tabs_on_click()

In [5]:
if __name__ == '__main__':
    set_course()


Dropdown(description='Course:', index=1, layout=Layout(width='max-content'), options=('ist256-spring2024', 'ts…

ToggleButtons(layout=Layout(width='max-content'), options=('Assignments', 'Roster'), tooltips=('View Assignmen…

Output()